# Dota 2 Data Collection & Processing
#### CSC4730 - Chandler Bone

This is a collection of my Dota 2 stats using the Opendota API. <br>
Dota 2 is an online MOBA game that is very complex in nature with currently 123 heroes to select from. <br>
This project will be a summarization of different stats I collect about myself in the currently 1400+ games I have played over the years. <br>

If you would like to use your own api key from here: https://docs.opendota.com/ <br>
Logging into your profile on https://www.opendota.com you can get your player_id from the url.

In [2]:
import config

import requests
import pandas as pd
import json

### Initialize our connection to the API

In [3]:
result = requests.get(f"https://api.opendota.com/api/matches/271145478?api_key={config.api_key}")
result.status_code

200

### Request our matches

In [4]:
matches = requests.get(f"https://api.opendota.com/api/players/{config.player_id}/matches")
matches_df = pd.read_json(matches.text)
matches_df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,deaths,assists,skill,leaver_status,party_size
0,6544535225,3,True,1710,22,7,128,2022-04-28 02:03:40,21.0,3,1,17,NaN,0,3.0
1,6544383633,128,True,2680,22,7,128,2022-04-27 21:26:56,21.0,2,5,6,NaN,0,2.0
2,6543173671,129,False,3052,22,7,129,2022-04-27 03:30:58,NaN,6,8,15,NaN,0,3.0
3,6543147508,3,True,1917,22,7,128,2022-04-27 02:49:44,NaN,13,1,11,NaN,0,3.0
4,6543128032,130,False,1368,22,7,129,2022-04-27 02:18:51,NaN,2,1,10,NaN,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,1212863940,2,True,3333,1,0,35,2015-02-04 23:37:37,NaN,5,16,16,NaN,0,NaN
1486,1191146511,131,True,2452,1,0,35,2015-01-27 00:13:37,NaN,1,7,4,NaN,0,NaN
1487,866365677,129,True,1947,1,0,5,2014-08-31 03:09:05,NaN,0,5,0,NaN,0,NaN
1488,743658022,2,False,1575,1,0,67,2014-06-26 23:37:19,NaN,1,7,1,NaN,1,NaN


### Requesting the hero ID's so we can merge them with the matches dataframe.

In [5]:
heroes = requests.get("https://api.opendota.com/api/constants/heroes")
heroes_df = pd.read_json(heroes.text).T
heroes_df.rename(columns={'id':'hero_id', 'localized_name':'hero_name'}, inplace=True)
heroes_df[:5]

,hero_id,name,hero_name,primary_attr,attack_type,roles,img,icon,base_health,base_health_regen,...,str_gain,agi_gain,int_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,cm_enabled,legs
1,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",/apps/dota2/images/dota_react/heroes/antimage....,/apps/dota2/images/dota_react/heroes/icons/ant...,200,0.25,...,1.6,2.8,1.8,150,0,1.4,310,None,True,2
2,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",/apps/dota2/images/dota_react/heroes/axe.png?,/apps/dota2/images/dota_react/heroes/icons/axe...,200,2.75,...,3.4,2.2,1.6,150,900,1.7,310,None,True,2
3,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",/apps/dota2/images/dota_react/heroes/bane.png?,/apps/dota2/images/dota_react/heroes/icons/ban...,200,0.25,...,2.5,2.5,2.5,400,900,1.7,305,None,True,4
4,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",/apps/dota2/images/dota_react/heroes/bloodseek...,/apps/dota2/images/dota_react/heroes/icons/blo...,200,0.25,...,2.7,3.1,2,150,900,1.7,300,None,True,2
5,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",/apps/dota2/images/dota_react/heroes/crystal_m...,/apps/dota2/images/dota_react/heroes/icons/cry...,200,0.25,...,2.2,1.6,3.3,600,900,1.7,280,None,True,2


In [6]:
matches_df = pd.merge(matches_df, heroes_df[['hero_id','hero_name']], how='left')

### Player wins arent explicitly given so we figure which side we're on and who won.

In [7]:
matches_df['player_side'] = "Radiant"
matches_df.loc[matches_df.player_slot >= 128, ['player_side']] = "Dire"

matches_df['player_win'] = True
matches_df.loc[(matches_df['player_side'] == "Dire") & (matches_df['radiant_win'] == True), ['player_win']] = False
matches_df.loc[(matches_df['player_side'] == "Radiant") & (matches_df['radiant_win'] == False), ['player_win']] = False
matches_df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,deaths,assists,skill,leaver_status,party_size,hero_name,player_side,player_win
0,6544535225,3,True,1710,22,7,128,2022-04-28 02:03:40,21.0,3,1,17,NaN,0,3.0,Snapfire,Radiant,True
1,6544383633,128,True,2680,22,7,128,2022-04-27 21:26:56,21.0,2,5,6,NaN,0,2.0,Snapfire,Dire,False
2,6543173671,129,False,3052,22,7,129,2022-04-27 03:30:58,NaN,6,8,15,NaN,0,3.0,Mars,Dire,True
3,6543147508,3,True,1917,22,7,128,2022-04-27 02:49:44,NaN,13,1,11,NaN,0,3.0,Snapfire,Radiant,True
4,6543128032,130,False,1368,22,7,129,2022-04-27 02:18:51,NaN,2,1,10,NaN,0,2.0,Mars,Dire,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,1212863940,2,True,3333,1,0,35,2015-02-04 23:37:37,NaN,5,16,16,NaN,0,NaN,Sniper,Radiant,True
1486,1191146511,131,True,2452,1,0,35,2015-01-27 00:13:37,NaN,1,7,4,NaN,0,NaN,Sniper,Dire,False
1487,866365677,129,True,1947,1,0,5,2014-08-31 03:09:05,NaN,0,5,0,NaN,0,NaN,Crystal Maiden,Dire,False
1488,743658022,2,False,1575,1,0,67,2014-06-26 23:37:19,NaN,1,7,1,NaN,1,NaN,Spectre,Radiant,False


### Get day of the week the game was on.

In [8]:
matches_df['day_of_week'] = matches_df['start_time'].dt.day_name()

### Export

In [9]:
matches_df.to_csv("matches.csv")

<br><br>
## Collecting Recent Match Data
#### (This data has more specific data points, but less collected data)

In [15]:
rc_matches = requests.get(f"https://api.opendota.com/api/players/{config.player_id}/recentMatches")
rc_matches_df = pd.read_json(rc_matches.text)
rc_matches_df= pd.merge(rc_matches_df, heroes_df[['hero_id','hero_name']], how='left')

In [19]:
rc_matches_df['player_side'] = "Radiant"
rc_matches_df.loc[rc_matches_df.player_slot >= 128, ['player_side']] = "Dire"

rc_matches_df['player_win'] = True
rc_matches_df.loc[(rc_matches_df['player_side'] == "Dire") & (rc_matches_df['radiant_win'] == True), ['player_win']] = False
rc_matches_df.loc[(rc_matches_df['player_side'] == "Radiant") & (rc_matches_df['radiant_win'] == False), ['player_win']] = False
rc_matches_df[:5]

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,...,last_hits,lane,lane_role,is_roaming,cluster,leaver_status,party_size,hero_name,player_side,player_win
0,6544535225,3,True,1710,22,7,128,2022-04-28 02:03:40,21.0,3,...,133,3.0,3.0,0.0,274,0,3,Snapfire,Radiant,True
1,6544383633,128,True,2680,22,7,128,2022-04-27 21:26:56,21.0,2,...,284,1.0,3.0,0.0,273,0,2,Snapfire,Dire,False
2,6543173671,129,False,3052,22,7,129,2022-04-27 03:30:58,NaN,6,...,165,NaN,NaN,NaN,193,0,3,Mars,Dire,True
3,6543147508,3,True,1917,22,7,128,2022-04-27 02:49:44,NaN,13,...,214,NaN,NaN,NaN,273,0,3,Snapfire,Radiant,True
4,6543128032,130,False,1368,22,7,129,2022-04-27 02:18:51,NaN,2,...,97,NaN,NaN,NaN,274,0,2,Mars,Dire,True


In [20]:
rc_matches_df.columns

Index(['match_id', 'player_slot', 'radiant_win', 'duration', 'game_mode',
       'lobby_type', 'hero_id', 'start_time', 'version', 'kills', 'deaths',
       'assists', 'skill', 'xp_per_min', 'gold_per_min', 'hero_damage',
       'tower_damage', 'hero_healing', 'last_hits', 'lane', 'lane_role',
       'is_roaming', 'cluster', 'leaver_status', 'party_size', 'hero_name',
       'player_side', 'player_win'],
      dtype='object')

In [21]:
rc_matches_df.to_csv("recent_matches.csv")